<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo5/2_drifts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1hiUFVbQ2Jgrv0olU-pcf89ODJ5t2pRVe" width="100%">

# Monitoreo de Modelos
---

En este notebook veremos una introducción al monitoreo de modelos desplegados por medio de técnicas de detección de drifts. Para esto comenzamos importando las librerías necesarias:

In [ ]:
import numpy as np
from IPython.display import display

## **1. Drifts en Machine Learning**
---

En el contexto de Machine Learning, el término "drift" se refiere a un cambio en la relación entre las variables de entrada y la variable objetivo del modelo a lo largo del tiempo. Este cambio puede ser causado por diferentes factores, como cambios en los datos de entrada o cambios en el entorno en el que el modelo está operando. Veamos los tipos de drifts que normalmente se deben identificar:

- **Data drift**: este tipo de drift se presenta cuando hay algún cambio notorio tanto en los datos de entrada del modelo (feature drift) como en el vector de etiquetas (label drift).

  <img src="https://drive.google.com/uc?export=view&id=1h3IZ_suNwsCN2HaTuWiAvxCasEKAbnOF" width="80%">

- **Concept drift**: este tipo de drift se presenta cuando no hay cambio en los datos de entrada del modelo pero si en la relación de los mismos con la etiqueta.

  <img src="https://drive.google.com/uc?export=view&id=1S2ffPPYifKR8Yb2W1wUZMx2DFh_qIXc4" width="80%">

- **Virtual drift**: este drift se presenta cuando un mismo modelo se puede aplicar a pesar de que la relación de los datos ha cambiado, por ejemplo, cuando los datos se separan un poco de la frontera de decisión en modelos de clasificación:

  <img src="https://drive.google.com/uc?export=view&id=1-T4-B1K8spQ36uYbbz0wI4Ia8zkZnN5Z" width="80%">

Existen diversas formas de detectar **drifts** en _Python_, no obstante, en este caso veremos cómo podemos identificarlos con la herramienta `evidently`.

## **2. Evidently**
---

Evidently es una biblioteca de Python de código abierto que se utiliza para la validación y el monitoreo de modelos de Machine Learning. Esta biblioteca proporciona herramientas para comparar la distribución de los datos de entrenamiento y los datos de prueba, evaluar el desempeño de los modelos, analizar los errores del modelo y monitorear los cambios en el rendimiento del modelo a lo largo del tiempo.

<img src="https://drive.google.com/uc?export=view&id=1YqkZCfLvgAMmpm23mSPKmOO4qX_fndcu" width="60%">

La biblioteca Evidently permite a los usuarios visualizar y analizar métricas de modelos como la precisión, el sesgo, la varianza, la distribución de errores, entre otras. Además, Evidently proporciona herramientas para visualizar la importancia de las características y las relaciones entre las características y la variable objetivo. Evidently es una herramienta útil para validar y monitorear el desempeño de los modelos de Machine Learning en diferentes etapas del ciclo de vida de un modelo, lo que puede ayudar a mejorar la confianza en los resultados del modelo y a detectar problemas o drifts en el modelo.

Veamos cómo instalarla:

In [ ]:
!pip install evidently

## **3. Conjunto de Datos**
---

En este caso estaremos trabajando sobre el conjunto de datos "California Housing" es uno de los conjuntos de datos más utilizados en Machine Learning. Este conjunto de datos contiene información sobre los precios de la vivienda y otras características en diferentes regiones de California, Estados Unidos. El conjunto de datos se compone de `20640` observaciones, cada una de las cuales representa una región de California.

<center><img src="https://drive.google.com/uc?export=view&id=1hrXXCboMcy_gXdj3FIF5qPjGdCgoL2gV" width="80%"></center>

Veamos cómo cargarlo desde `sklearn`:

In [ ]:
from sklearn.datasets import fetch_california_housing

Cargamos el conjunto de datos:

In [ ]:
dataset = fetch_california_housing(as_frame=True)["frame"]
display(dataset)

Este conjunto de datos tiene los siguientes campos:

- `MedInc`: ingreso mediano de los hogares en la región.
- `HouseAge`: edad mediana de las casas en la región.
- `AveRooms`: número promedio de habitaciones por vivienda en la región.
- `AveBedrms`: número promedio de dormitorios por vivienda en la región.
- `Population`: población total de la región.
- `AveOccup`: número promedio de personas que ocupan una vivienda en la región.
- `Latitude`: latitud geográfica de la región.
- `Longitude`: longitud geográfica de la región.

In [ ]:
print(dataset.columns)

El objetivo de este conjunto de datos es estimar el valor de `MedHouseVal`, por medio de algún modelo de regresión. Vamos a renombrar esta columna como `target`:

In [ ]:
dataset = dataset.rename(columns={"MedHouseVal": "target"})

Ahora, vamos a simular los datos predichos por un modelo al tomar los datos originales y agregando un poco de ruido normal:

In [ ]:
dataset = dataset.assign(
    prediction = dataset.target + np.random.normal(0, 3, size=(dataset.shape[0]))
)
display(dataset.head())

Ahora, vamos a generar dos versiones del conjunto de datos para simular dos muestras tomadas en instantes distintos.

> **Nota**: recuerde que el método `sample` en `pandas` nos permite obtener una muestra aleatoria de un `DataFrame`.:

In [ ]:
reference = dataset.sample(n=5000, replace=False)
current = dataset.sample(n=5000, replace=False)

In [ ]:
reference

In [ ]:
current

## **4. Reportes**
---

El uso de `evidently` consiste en la creación de un reporte al que se le agregan distintas métricas y evaluaciones, comenzaremos importando la clase `Report`:

In [ ]:
from evidently.report import Report

Ahora, importaremos un grupo de métricas típicas para la evaluación de data drift con la clase `DataDriftPreset`:

In [ ]:
from evidently.metric_preset import DataDriftPreset

Creamos el reporte:

In [ ]:
report = Report(
        metrics=[DataDriftPreset()]
        )

Ejecutamos el reporte pasandole las dos versiones del conjunto de datos:

In [ ]:
report.run(reference_data=reference, current_data=current)

Por último mostramos el reporte:

In [ ]:
display(report)

Este preset nos permitirá evidenciar si alguna variable ha cambiado considerablemente con respecto al valor de referencia. El tablero le permitirá observar las distribuciones de los datos y una métrica (la distancia de Wasserstein) que permite identificar si alguna variable ha tenido algún cambio estadísticamente considerable.

En algunas oportunidades queremos enfocarnos en una variable en específico, esto lo podemos lograr con métricas de columna como las que se muestran a continuación:

In [ ]:
from evidently.metrics import ColumnDriftMetric, ColumnSummaryMetric, ColumnQuantileMetric

Veamos cómo generar un reporte detallado únicamente para la columna `AveRooms`:

In [ ]:
report = Report(metrics=[
    ColumnSummaryMetric(column_name='AveRooms'), # resumen de la variable
    ColumnQuantileMetric(column_name='AveRooms', quantile=0.25), # cuantil 0.25
    ColumnDriftMetric(column_name='AveRooms'), # detección de drifts

])

report.run(reference_data=reference, current_data=current)
display(report)

## **5. Tests**
---

Con `evidently` también es posible realizar pruebas de calidad de datos y predicciones. Entre las cosas que podemos identificar encontramos:

- **Valores faltantes**: permite identificar la aparición de valores faltantes tanto en filas como columnas.
- **Columnas constantes**: permite identificar si alguna columna tiene un valor que nunca cambia.
- **Registros duplicados**: permite ver si hay duplicidad en los datos.
- **Tipos de columnas**: valida si el tipo de una variable ha cambiado.

Para armar un reporte de tipo test debemos utilizar un `TestSuite`, vamos a importarlo:

In [ ]:
from evidently.test_suite import TestSuite

Al igual que con los drift, también disponemos de algunos test generales como el `NoTargetPerformanceTestPreset`, el cual nos permite realizar pruebas sobre el conjunto de datos sin la etiqueta:

In [ ]:
from evidently.test_preset import NoTargetPerformanceTestPreset

Veamos el test, la sintaxis es equivalente a la de los drifts:

In [ ]:
suite = TestSuite(tests=[
    NoTargetPerformanceTestPreset(),
    ])

suite.run(reference_data=reference, current_data=current)
display(suite)

Adicionalmente, también podemos realizar tests específicos, primero los importamos:

In [ ]:
from evidently.tests import (
        TestNumberOfColumnsWithMissingValues,
        TestNumberOfRowsWithMissingValues,
        TestNumberOfConstantColumns,
        TestNumberOfDuplicatedRows,
        TestNumberOfDuplicatedColumns,
        TestColumnsType,
        TestNumberOfDriftedColumns
        )

Veamos cómo podemos aplicarlo:

In [ ]:
suite = TestSuite(tests=[
    TestNumberOfColumnsWithMissingValues(),
    TestNumberOfRowsWithMissingValues(),
    TestNumberOfConstantColumns(),
    TestNumberOfDuplicatedRows(),
    TestNumberOfDuplicatedColumns(),
    TestColumnsType(),
    TestNumberOfDriftedColumns()
    ])

suite.run(reference_data=reference, current_data=current)
display(suite)

Por último, veamos la detección de algunos tipos de drifts.

Primero generemos un conjunto de datos con variaciones en las características para detectar **feature drift**, para ello, vamos a asignar la columna `MedInc` como un valor constante:

In [ ]:
reference = dataset.sample(n=5000, replace=False)
current = (
        dataset
        .sample(n=5000, replace=False)
        .assign(MedInc=500)
        )

Ahora, generamos un reporte de drifts de datos:

In [ ]:
report = Report(
        metrics=[DataDriftPreset()]
        )
report.run(reference_data=reference, current_data=current)
display(report)

También podemos detectar **label drift**, para ello, vamos a modificar la columna `target`:

In [ ]:
reference = dataset.sample(n=5000, replace=False)
current = (
        dataset
        .sample(n=5000, replace=False)
        .assign(target=300)
        )

Para detectar **label drift** podemos usar la clase `TargetDriftPreset`, veamos cómo importarla:

In [ ]:
from evidently.metric_preset import TargetDriftPreset

Creamos el reporte:

In [ ]:
report = Report(
        metrics=[TargetDriftPreset()]
        )
report.run(reference_data=reference, current_data=current)
display(report)

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [Data Drift Algorithm](https://docs.evidentlyai.com/reference/data-drift-algorithm)
- [Analyze Target and Prediction Drift in Machine Learning Models](https://www.evidentlyai.com/blog/evidently-014-target-and-prediction-drift)
- [Why you should care about data and concept drift](https://www.evidentlyai.com/blog/machine-learning-monitoring-data-and-concept-drift)

## Créditos
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Asistente docente**:

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Coordinador de virtualización:**

- [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Diseño de imágenes:**
  - [Rosa Alejandra Superlano Esquibel](https://www.linkedin.com/in/alejandra-superlano-02b74313a/).
  - [Mario Andrés Rodríguez Triana](mailto:mrodrigueztr@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*